In [ ]:
from binance import Client
import pandas as pd
import asyncio
import sqlalchemy
import sched, time
from binance import BinanceSocketManager
import numpy
import sqlite3

In [ ]:
#API KEY TO CONNECT TO BINANCE_TEST_NET
api_key = '8Y2R2t7fmc9NIQWiR2dusAoM27fBrlm0BWU8FupEYcUO1rnY49RvLLRt0FoTa60U'
api_secret = '6HHeNMymyghzVVhBKopjZMIsdVCyPwGbzhryJmDZAeMmzdvX4pXdwYkLpCXUUXS4'

In [ ]:
#LOAD BINANCE CLIENT + SOCKETMANAGER
client = Client(api_key, api_secret, testnet=True)
bsm = BinanceSocketManager(client)

In [ ]:
#CREATE DATAFRAME FOR PRICE DATA
def createframe(msg): 
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']]
    df.columns = ['symbol','Time','Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [ ]:
#LOAD SQL ENGINE / CONNECT TO SQL DATABASE
engine_2 = sqlalchemy.create_engine('sqlite:///TraidsUSDTBUSD.db')

In [ ]:
#GET HISTORICAL DATA FOR (SYMBOL / INTERVAL/ LAST X MIN )
pd.DataFrame(client.get_historical_klines('BTCUSDT', '1m','30 min ago UTC'))

In [ ]:
#DATAFRAME FOR HISTORICAL DATA
def getminutedata(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval,lookback +'min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time','Open','Hight','Low','Close','Volume']
    frame= frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame



In [ ]:
#DATAFRAME FOR EXECUTED ORDERS 
def createframe(order): 
    df_2 = pd.DataFrame([order])
    df_2 = df_2.loc[:,['symbol', 'side', 'type', 'status','executedQty','cummulativeQuoteQty','PnL','transactTime']]
    df_2.columns = ['symbol', 'side','type', 'status','executedQty','cummulativeQuoteQty','PnL','transactTime']
    df_2.transactTime= df_2.transactTime.astype(float)
    return df_2

In [ ]:
#DATAFRAME WILL BE STORED IN SQL FILE THROUGH THE ENGINE
def orderToSql(order): 
    frame = createframe(order)
    frame.to_sql('TraidsUSDTBUSD', engine_2, if_exists='append', index= False) 

In [ ]:
#TRADING STRATEGIE & BOT 
#SIMPLE TREND FOLLOWING - IF PRICE RISES FOR TIME X THE BOT WILL PLACE AN ORDER AND SELLS IF PRICE IS HIGHER (TO GAIN PROFIT) OR LOWER (TO CUT LOSS) THAN X% 
async def strategytest(symbol, qty, entried=False):
    df = getminutedata(symbol,'1m','30')
    cumulret = (df.Open.pct_change() +1).cumprod()-1
    socket = bsm.trade_socket(symbol)
    if not entried:
            if cumulret[-1]< -0.0002:
                order = client.create_order(symbol = symbol, side ='BUY', type='MARKET', quantity = qty)
                PnL = '0'
                order['PnL'] = PnL
                buyprice = float(order['fills'][0]['price'])
                orderToSql(order)
                print(order)
                entried=True
        
            else:
                print('no trade has been executed')
                
    if entried:
        while True: 
            await socket.__aenter__()
            df =  getminutedata(symbol,'1m','30')
            sincebuy = df.loc[df.index > pd.to_datetime(order ['transactTime'],unit='ms')]
            if len(sincebuy) > 0:
                sincebuyret = (sincebuy.Open.pct_change() +1).cumprod()-1
                if sincebuyret[-1] > 0.0015 or sincebuyret[-1] < 0.0015:
                    order = client.create_order(symbol = symbol, side ='SELL', type='MARKET', quantity = qty)
                    sellprice = float(order['fills'][0]['price'])
                    PnL = (buyprice - sellprice)
                    order['PnL'] = PnL
                    len(order)     
                    orderToSql(order)
                    print(order)
            
                    break 

In [ ]:
#EXECUTION OF THE MAIN FUNCTION
try:
    while True:
        await strategytest('BTCUSDT', 0.001 )
except KeyboardInterrupt:
    print('interrupted!') 